In [30]:
import numpy as np
import sklearn.linear_model as model
import csv
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler

First, load in the training and test datasets

In [2]:
train = np.loadtxt('Data/training_data.txt', skiprows=1)
test = np.loadtxt('Data/test_data.txt', skiprows=1)

Isolate the labels from the features.

In [3]:
feature_cols = []
for i in range(1000):
    feature_cols.append(i)
    
X_feats = train[:, feature_cols]
X_test = test[:, feature_cols]
y_feats = train[:, 1000]

A helper function to compute the classification error:

In [4]:
def class_error(y, y_pred):
    misclassified = 0
    for i in range(len(y)):
        if y[i] != y_pred[i]:
            misclassified += 1
    return float(misclassified) / len(y)

In [18]:
minority_idxs = np.flatnonzero(y_feats == 1)
majority_idxs = np.flatnonzero(y_feats == 0)
idxs = np.hstack((majority_idxs, minority_idxs))
X = X_feats[idxs]
y = y_feats[idxs]
y[y == 0] = 0
y[y == 1] = 1

Shuffle the training set then split it into 80% training and 20% validation

Train the model and print the training error and validation error. Use SMOTE to fix oversampling

In [64]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_sample(X, y)

In [65]:
num_cols = []
for i in range(len(y_res)):
    num_cols.append(i)

trainCutoff = int(0.8 * len(y_res))

In [66]:
np.random.shuffle(num_cols)
train_indices = num_cols[:trainCutoff]
valid_indices = num_cols[trainCutoff:]
X_train = np.take(X_res, train_indices, axis=0)
y_train = np.take(y_res, train_indices, axis=0)
X_valid = np.take(X_res, valid_indices, axis=0)
y_valid = np.take(y_res, valid_indices, axis=0)

A function to write the prediction to a text file:

In [70]:
def writeToText(predictions):
    array = [["Id","Prediction"]]
    index = 1
    for i in range(len(predictions)):
        predict = predictions[index - 1]
        array.append([index, int(predictions[index - 1])])
        index += 1
    f = open("test_predictions2.txt", 'w')
    writer = csv.writer(f, delimiter=',', quotechar='|')
    writer.writerows(array)

In [68]:
logistic = model.LogisticRegression()
logistic.fit(X_train, y_train)

print(class_error(y_valid, logistic.predict(X_valid)))

0.013969292725899824


In [71]:
writeToText(logistic.predict(X_test))